# GSM model for E. coli

In [1]:
import numpy as np
from cobra.io import read_sbml_model

In [2]:
model = read_sbml_model('iML1515.xml.gz')

We want to look at the model. See if it has fluxes

In [3]:
solution = model.optimize()
solution.fluxes

CYTDK2     0.00000
XPPT       0.00000
HXPRT      0.00000
NDPK5      0.00000
SHK3Dr     0.33424
            ...   
MPTS       0.00000
MOCOS      0.00000
BMOGDS2    0.00000
FESD2s     0.00000
OCTNLL     0.00000
Name: fluxes, Length: 2712, dtype: float64

In [4]:
model.summary()

We want to find the metabolite Purtescine in the model

In [5]:
for metabolite in model.metabolites.query('Putrescine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

Putrescine
ptrc_p
Putrescine
ptrc_c
Putrescine
ptrc_e


In [6]:
model.metabolites.get_by_id('ptrc_c') # Putrescine

Metabolite identifier,ptrc_c
Name,Putrescine
Memory address,0x07fd0fe1e1da0
Formula,C4H14N2
Compartment,c
In 9 reaction(s),"PTRCORNt7pp, SPMS, PTRCTA, AGMT, BIOMASS_Ec_iML1515_WT_75p37M, PTRCabcpp, GGPTRCS, PTRCt2pp, ORNDC"


The reaction that produces purtescine from ornithine is native to E. coli

In [7]:
model.reactions.ORNDC

Reaction identifier,ORNDC
Name,Ornithine Decarboxylase
Memory address,0x07fd0fce067f0
Stoichiometry,h_c + orn_c --> co2_c + ptrc_c H+ + Ornithine --> CO2 CO2 + Putrescine
GPR,b2965 or b0693
Lower bound,0.0
Upper bound,1000.0


We want to check the flux

In [8]:
with model:
    model.objective = model.reactions.ORNDC
    print(model.optimize().objective_value)  # We produce Purtescine

41.09128205128201


In [9]:
with model:
    model.objective = model.reactions.EX_ptrc_e
    print(model.optimize().objective_value)  # We produce Purtescine.EX_ptrc_e

9.56477064220184


# Adding enzyme 2.6.1.13

In [10]:
from cobra import Reaction, Metabolite

To make E. Coli produce putrescine we need to add the reaction of the enzyme 2.6.1.13.
This is the same as we added in C. Glutamicum.

In [11]:
new_reaction_1 = Reaction('OAT')  # OAT -> (o)rnithine (a)mino(t)ransferase

In [12]:
for metabolite in model.metabolites.query('5-semialdehyde', 'name'):
    print(metabolite.name)
    print(metabolite.id)

N-Acetyl-L-glutamate 5-semialdehyde
acg5sa_c
N2-Succinyl-L-glutamate 5-semialdehyde
sucgsa_c
L-Glutamate 5-semialdehyde
glu5sa_c


In [13]:
model.metabolites.get_by_id('glu5sa_c') # L-glutamate 5-semialdehyde

Metabolite identifier,glu5sa_c
Name,L-Glutamate 5-semialdehyde
Memory address,0x07fd0fe37be80
Formula,C5H9NO3
Compartment,c
In 2 reaction(s),"G5SD, G5SADs"


We see that the two reactions L-glutamate 5-semialdehyde is part of correspond to the ones seen in the KEGG pathway map. We now need to find the other metabolites that play a role in conversion of L-glutamate 5-semialdehyde to ornithine.

In [14]:
for metabolite in model.metabolites.query('Ornithine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

Ornithine
orn_e
Ornithine
orn_p
Ornithine
orn_c


In [15]:
model.metabolites.get_by_id('orn_c') # Ornithine

Metabolite identifier,orn_c
Name,Ornithine
Memory address,0x07fd0fe296828
Formula,C5H13N2O2
Compartment,c
In 6 reaction(s),"OCBT, ORNabcpp, PTRCORNt7pp, ARGORNt7pp, ORNDC, ACODA"


In [16]:
for metabolite in model.metabolites.query('Oxoglutarate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

2-Oxoglutarate
akg_e
2-Oxoglutarate
akg_c
2-Oxoglutarate
akg_p


In [17]:
model.metabolites.get_by_id('akg_c') # 2-Oxoglutarate

Metabolite identifier,akg_c
Name,2-Oxoglutarate
Memory address,0x07fd0fe5dc518
Formula,C5H4O5
Compartment,c
In 28 reaction(s),"TYRTA, ICDHyr, VALTA, OHPBAT, CYSTA, AHGDx, ABTA, ILETA, TDPAGTA, ACOTA, AKGDH, PHETA1, SDPTA, UDPKAAT, SHGO, PTRCTA, ALATA_L, SOTA, AKGt2rpp, ARHGDx, LEUTAi, PSERT, HSTPT, ASPTA, GLUSy, GLUDy, TAU..."


In [18]:
for metabolite in model.metabolites.query('L-Glutamate', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-Glutamate 1-semialdehyde
glu1sa_c
L-Glutamate
glu__L_c
L-Glutamate 5-semialdehyde
glu5sa_c
L-Glutamate
glu__L_e
L-Glutamate
glu__L_p
L-Glutamate 5-phosphate
glu5p_c


In [19]:
model.metabolites.get_by_id('glu__L_c') # L-Glutamate

Metabolite identifier,glu__L_c
Name,L-Glutamate
Memory address,0x07fd0fe377e10
Formula,C5H8NO4
Compartment,c
In 51 reaction(s),"GGGABAH, GLUABUTt7pp, IG3PS, GLUPRT, ILETA, GGPTRCS, GF6PTA, LALGP, CBPS, PHETA1, GLUt4pp, PTRCTA, ACGS, ADCS, GLUabcpp, PSERT, GLUTRS, GLUSy, GLNS, GLUN, SGDS, PRFGS, GLUR, TYRTA, VALTA, GLU5K, GM..."


Now we add the reaction to produce ornithine

In [20]:
new_reaction_1.add_metabolites({model.metabolites.get_by_id('glu5sa_c'): -1, # L-glutamate 5-semialdehyde
                              model.metabolites.get_by_id('glu__L_c'): -1, # L-glutamate 
                              model.metabolites.get_by_id('orn_c'): 1, # L-ornithine
                              model.metabolites.get_by_id('akg_c'): 1 # 2-oxoglutarate
                             })

In [21]:
print(new_reaction_1.build_reaction_string())

glu5sa_c + glu__L_c --> akg_c + orn_c


In [22]:
model.add_reactions([new_reaction_1])
model.reactions.OAT

Reaction identifier,OAT
Name,
Memory address,0x07fd0fd089048
Stoichiometry,glu5sa_c + glu__L_c --> akg_c + orn_c L-Glutamate 5-semialdehyde + L-Glutamate --> 2-Oxoglutarate + Ornithine
GPR,
Lower bound,0.0
Upper bound,1000.0


In [23]:
with model:
    model.objective = model.reactions.OAT
    print(model.optimize().objective_value)  # We produce L-ornithine

53.02333333333331


We want to see production rate of putrescine now after we have added a new path via glutamate

In [24]:
with model:
    model.objective = model.reactions.ORNDC
    print(model.optimize().objective_value)  # We produce Purtescine

45.787428571428464


In [25]:
with model:
    model.objective = model.reactions.EX_ptrc_e
    print(model.optimize().objective_value)  # We produce Purtescine

9.92914285714285


We see that the production rate of putrescine increases from 41.09 to 45.79

# Adding enzyme 3.5.3.1

Uses L-arginine and H2O to make L-ornithine and Urea

In [26]:
for metabolite in model.metabolites.query('L-Arginine', 'name'):
    print(metabolite.name)
    print(metabolite.id)

L-Arginine
arg__L_c
L-Arginine
arg__L_e
L-Arginine
arg__L_p


In [27]:
for metabolite in model.metabolites.query('Urea', 'name'):
    print(metabolite.name)
    print(metabolite.id)

Urea CH4N2O
urea_c
Urea CH4N2O
urea_p
Urea CH4N2O
urea_e


In [28]:
for metabolite in model.metabolites.query('H2O', 'name'):
    print(metabolite.name)
    print(metabolite.id)

H2O H2O
h2o_p
H2O H2O
h2o_c
H2O H2O
h2o_e


In [29]:
new_reaction_2 = Reaction('AAH')  # AAH -> L-arginine amidinohydrolase

In [30]:
new_reaction_2.add_metabolites({model.metabolites.get_by_id('arg__L_c'): -1, # L-arginine
                              model.metabolites.get_by_id('h2o_c'): -1, # H2O 
                              model.metabolites.get_by_id('orn_c'): 1, # L-ornithine
                              model.metabolites.get_by_id('urea_c'): 1 # Urea
                             })

In [31]:
print(new_reaction_2.build_reaction_string())

arg__L_c + h2o_c --> orn_c + urea_c


In [32]:
model.add_reactions([new_reaction_2])
model.reactions.AAH

Reaction identifier,AAH
Name,
Memory address,0x07fd0fd066b38
Stoichiometry,arg__L_c + h2o_c --> orn_c + urea_c L-Arginine + H2O H2O --> Ornithine + Urea CH4N2O
GPR,
Lower bound,0.0
Upper bound,1000.0


In [33]:
with model:
    model.objective = model.reactions.ORNDC
    print(model.optimize().objective_value)  # We produce Purtescine

45.787428571428464


In [34]:
with model:
    model.objective = model.reactions.EX_ptrc_e
    print(model.optimize().objective_value)

9.929142857142857
